In [4]:
import pandas as pd
df = pd.read_csv("data.csv")
df = df.dropna()
df = df.sample(frac=1)
df.head()

,Unnamed: 0,message,category
34812,9636,And yet there is no need to visit anywhere?,0
4222,4222,It turns out we have every chance to become stars,0
33470,8294,about 100 times,0
12675,12675,​​AT&T Customer Files Case Against the Firm Fo...,1
22075,22075,A very long time ago I wrote a text about tona...,2


In [5]:
df_train = pd.read_csv("train_data.csv")
categories = pd.read_csv("train_solution.csv")
categories.head()
df_train["category"] = categories.category
df_train.head()
df = df.append(df_train)
df = df.drop(columns = ["id", "Unnamed: 0"])
df.head()

/tmp/ipykernel_34393/2305811941.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_train)


,message,category
34812,And yet there is no need to visit anywhere?,0
4222,It turns out we have every chance to become stars,0
33470,about 100 times,0
12675,​​AT&T Customer Files Case Against the Firm Fo...,1
22075,A very long time ago I wrote a text about tona...,2


In [6]:
print(len(df[(df.category == 0) & (df.message.str.len() > 20)]))
print(len(df[(df.category == 1) & (df.message.str.len() > 20)]))
print(len(df[(df.category == 2) & (df.message.str.len() > 20)]))

15564
8039
10573


In [7]:
mask = (df.message.str.len() > 20)
# df_balanced = df[mask]
# print(len(df_balanced))
# df_balanced.head()

In [8]:
import nltk
# !pip install inflect
import inflect
# !pip install contractions

import contractions
from bs4 import BeautifulSoup
import re, string, unicodedata
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder

In [9]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas
def normalize_text(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    #words = stem_words(words)
    words = lemmatize_verbs(words)
    return words

In [10]:
# Tokenize tweet into words
def tokenize(text):
    return nltk.word_tokenize(text)

In [11]:
def text_prepare(text):
    text = ' '.join([x for x in normalize_text(tokenize(text))])
    return text

In [12]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")
df['message'] = [text_prepare(x) for x in df['message']]
df.head()
df_balanced = df[mask]
df_balanced.head()

[nltk_data] Downloading package punkt to /home/andrey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/andrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/andrey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/andrey/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,message,category
34812,yet need visit anywhere,0
4222,turn every chance become star,0
12675,customer file case firm cause eighteen million...,1
22075,long time ago write text tonal products select...,2
7935,strangle everything dl write transformer,0


In [13]:
df_test = pd.read_csv("test_data.csv")
df_test.message = [text_prepare(x) for x in df_test.message]
X_global_test = df_test.message

In [14]:
from keras.layers import Dropout, Dense, Embedding, LSTM, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.metrics import matthews_corrcoef, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.utils import shuffle
import numpy as np
import pickle
import warnings
import logging

2022-10-20 21:21:46.369714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-20 21:21:46.369758: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B*.zip

In [15]:

def prepare_model_input(X_train, X_test, X_global_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.concatenate((text, X_global_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text) 
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train_Glove = text[0:len(X_train), ]
    X_test_Glove = text[len(X_train):len(X_test) + len(X_train), ]
    X_test_global_Glove = text[len(X_test) + len(X_train):, ]
    embeddings_dict = {}
    f = open("glove.6B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_dict[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_dict))
    return (X_train_Glove, X_test_Glove, X_test_global_Glove, word_index, embeddings_dict)

In [16]:
def build_bilstm(word_index, embeddings_dict, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5, hidden_layer = 3, lstm_node = 32):
    # Initialize a sequebtial model
    model = Sequential()
    # Make the embedding matrix using the embedding_dict
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_dict.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) != len(embedding_vector):
                print("could not broadcast input array from shape", str(len(embedding_matrix[i])),
                      "into shape", str(len(embedding_vector)), " Please make sure your"
                                                                " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)
            embedding_matrix[i] = embedding_vector
            
    # Add embedding layer
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    # Add hidden layers 
    for i in range(0,hidden_layer):
        # Add a bidirectional lstm layer
        model.add(Bidirectional(LSTM(lstm_node, return_sequences=True, recurrent_dropout=0.2)))
        # Add a dropout layer after each lstm layer
        model.add(Dropout(dropout))
    model.add(Bidirectional(LSTM(lstm_node, recurrent_dropout=0.2)))
    model.add(Dropout(dropout))
    # Add the fully connected layer with 256 nurons and relu activation
    model.add(Dense(256, activation='relu'))
    # Add the output layer with softmax activation since we have 2 classes
    model.add(Dense(nclasses, activation='softmax'))
    # Compile the model using sparse_categorical_crossentropy
    model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model

In [64]:
X = df.message
y = df.category
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)
print("Preparing model input ...")
X_train_Glove, X_test_Glove, X_global_test_Glove, word_index, embeddings_dict = prepare_model_input(X_train,X_test, X_global_test)
print("Done!")
print("Building Model!")
model = build_bilstm(word_index, embeddings_dict, 3)
model.summary()

Preparing model input ...
Found 49865 unique tokens.
(44964, 500)
Total 400000 word vectors.
Done!
Building Model!
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 500, 50)           2493300   
                                                                 
 bidirectional_24 (Bidirecti  (None, 500, 64)          21248     
 onal)                                                           
                                                                 
 dropout_24 (Dropout)        (None, 500, 64)           0         
                                                                 
 bidirectional_25 (Bidirecti  (None, 500, 64)          24832     
 onal)                                                           
                                                                 
 dropout_25 (Dropout)        (None, 500, 64)           0         
     

In [17]:
x_balance = df_balanced.message
y_balance = df_balanced.category
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balances = train_test_split(x_balance, y_balance, test_size = 0.05)
balanced_train, balance_test, balance_global_test, word_index_balance, embeddings_dict_balance = prepare_model_input(X_train_balanced,X_test_balanced, X_global_test)

Found 48223 unique tokens.
(40103, 500)
Total 400000 word vectors.


In [18]:
load_model = build_bilstm(word_index_balance, embeddings_dict_balance, 3)
# load_model.load_weights("train")
# load_model.summary()

2022-10-20 21:22:02.912176: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-20 21:22:02.912240: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-20 21:22:02.912303: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (andrey): /proc/driver/nvidia/version does not exist
2022-10-20 21:22:02.912839: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
import tensorflow as tf
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="exp3/train",
                                                 save_weights_only=True,
                                                 verbose=1)
history = load_model.fit(balanced_train, y_train_balanced,
                           validation_data=(balance_test,y_test_balances),
                           epochs=5,
                           batch_size=128,
                           callbacks=[cp_callback])

Epoch 1/5
254/254 [==============================] - ETA: 0s - loss: 0.3603 - accuracy: 0.8673
Epoch 1: saving model to exp3/train
254/254 [==============================] - 1040s 4s/step - loss: 0.3603 - accuracy: 0.8673 - val_loss: 0.1687 - val_accuracy: 0.9374
Epoch 2/5
254/254 [==============================] - ETA: 0s - loss: 0.1836 - accuracy: 0.9402
Epoch 2: saving model to exp3/train
254/254 [==============================] - 968s 4s/step - loss: 0.1836 - accuracy: 0.9402 - val_loss: 0.1476 - val_accuracy: 0.9473
Epoch 3/5
254/254 [==============================] - ETA: 0s - loss: 0.1350 - accuracy: 0.9560
Epoch 3: saving model to exp3/train
254/254 [==============================] - 971s 4s/step - loss: 0.1350 - accuracy: 0.9560 - val_loss: 0.1269 - val_accuracy: 0.9596
Epoch 4/5
254/254 [==============================] - ETA: 0s - loss: 0.1078 - accuracy: 0.9652
Epoch 4: saving model to exp3/train
254/254 [==============================] - 974s 4s/step - loss: 0.1078 - accura

In [14]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="train",
                                                 save_weights_only=True,
                                                 verbose=1)
history = model.fit(X_train_Glove, y_train,
                           validation_data=(X_test_Glove,y_test),
                           epochs=5,
                           batch_size=128,
                           callbacks=[cp_callback]
                           )
# model.save("nlp_model")
# model.fit()

Epoch 1/5
319/319 [==============================] - ETA: 0s - loss: 0.3265 - accuracy: 0.8809
Epoch 1: saving model to train
319/319 [==============================] - 1420s 4s/step - loss: 0.3265 - accuracy: 0.8809 - val_loss: 0.1961 - val_accuracy: 0.9403
Epoch 2/5
319/319 [==============================] - ETA: 0s - loss: 0.1787 - accuracy: 0.9436
Epoch 2: saving model to train
319/319 [==============================] - 1272s 4s/step - loss: 0.1787 - accuracy: 0.9436 - val_loss: 0.1634 - val_accuracy: 0.9492
Epoch 3/5
319/319 [==============================] - ETA: 0s - loss: 0.1403 - accuracy: 0.9564
Epoch 3: saving model to train
319/319 [==============================] - 1451s 5s/step - loss: 0.1403 - accuracy: 0.9564 - val_loss: 0.1512 - val_accuracy: 0.9538
Epoch 4/5
319/319 [==============================] - ETA: 0s - loss: 0.1110 - accuracy: 0.9650
Epoch 4: saving model to train
319/319 [==============================] - 1575s 5s/step - loss: 0.1110 - accuracy: 0.9650 - val_

In [15]:
model.predict(X_global_test_Glove)

array([[4.61483374e-02, 1.06141553e-03, 9.52790201e-01],
       [9.98336673e-01, 8.83608300e-06, 1.65443844e-03],
       [9.97342646e-01, 1.78258942e-05, 2.63948971e-03],
       ...,
       [9.97927904e-01, 1.59022129e-05, 2.05619377e-03],
       [1.03239385e-04, 1.40386064e-05, 9.99882698e-01],
       [9.90842104e-01, 5.18996494e-05, 9.10607539e-03]], dtype=float32)

In [22]:
prediction = load_model.predict(balance_global_test)
prediction

array([[4.4575298e-01, 1.7972300e-03, 5.5244982e-01],
       [9.9382621e-01, 8.7574124e-05, 6.0861930e-03],
       [9.9753654e-01, 2.2955353e-05, 2.4404943e-03],
       ...,
       [9.9736041e-01, 3.6472236e-05, 2.6031053e-03],
       [1.1726107e-03, 2.7282062e-05, 9.9880004e-01],
       [9.6360141e-01, 8.2724087e-04, 3.5571303e-02]], dtype=float32)

In [78]:
load_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 500, 50)           2471150   
                                                                 
 bidirectional_36 (Bidirecti  (None, 500, 64)          21248     
 onal)                                                           
                                                                 
 dropout_36 (Dropout)        (None, 500, 64)           0         
                                                                 
 bidirectional_37 (Bidirecti  (None, 500, 64)          24832     
 onal)                                                           
                                                                 
 dropout_37 (Dropout)        (None, 500, 64)           0         
                                                                 
 bidirectional_38 (Bidirecti  (None, 500, 64)         

In [82]:
len(X_global_test_Glove)

5927

In [20]:
prediction = load_model.predict(X_global_test_Glove)

NameError: name 'X_global_test_Glove' is not defined

In [23]:
predictions = list()
for line in prediction:
    try:
        one_pred = list(line).index(max(line))
    except:
        one_pred = line
    predictions.append(one_pred)
print(predictions[:10])

[2, 0, 0, 0, 2, 0, 1, 0, 0, 0]


In [25]:
predictions

[2,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,


In [24]:
df_result = pd.DataFrame()
print(len(predictions))
print(len(df_test.id))
df_result["id"] = df_test.id
df_result["category"] = predictions
df_result.head()


5927
5927


,id,category
0,275672,2
1,275673,0
2,275674,0
3,275675,0
4,275676,2


In [25]:
compression_opts = dict(method='zip',
                        archive_name='out4.csv')
df_result.to_csv('out4.zip', index=False,
          compression=compression_opts) 